In [1]:
pip install tensorflow-metal

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import os
import sys
from src.cleaning import datacleaning
from src import data_generation
from src.preparation import model_perform
import pandas as pd
import numpy as np
import aif360
from sklearn.model_selection import train_test_split
from aif360.sklearn.metrics import statistical_parity_difference
from aif360.sklearn.metrics import average_odds_difference
from aif360.sklearn.metrics import equal_opportunity_difference

Index(['complaint_id', 'complainant_age_incident', 'allegation',
       'contact_reason', 'substantiated', 'complainant_gender'],
      dtype='object')
Index(['complaint_id', 'complainant_age_incident', 'allegation',
       'contact_reason', 'substantiated', 'complainant_gender'],
      dtype='object')


/Users/alecpanattoni/Documents/MissingnessFairnessAnalysis/src/data_generation.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column].iloc[i] = np.nan
/Users/alecpanattoni/Documents/MissingnessFairnessAnalysis/src/data_generation.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[miss_column].iloc[i] = np.nan
/Users/alecpanattoni/Documents/MissingnessFairnessAnalysis/src/data_generation.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

,0,1,2,3,4,5,6,7,8,9,...,193,194,195,196,197,198,199,200,201,202
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


[['Female' 24.0 'Physical force' 'C/V at PCT to obtain information']
 ['Male' 44.0 'Physical force'
  'PD suspected C/V of violation/crime - bldg']
 ['Male' 49.0 'Handcuffs too tight'
  'PD suspected C/V of violation/crime - street']
 ...
 ['Male' 30.0 'Vehicle search' 'Moving violation']
 ['Male' 17.0 'Word' 'PD suspected C/V of violation/crime - street']
 ['Male' 46.0 'Physical force' 'Other']]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
data = datacleaning.cleaning(os.path.join(os.path.dirname(
    os.path.realpath('run.py')) + '/data/allegations_raw.csv'))

In [ ]:
target = sys.argv[1]

if target == "test":
    data = datacleaning.cleaning(os.path.join(os.path.dirname(
        os.path.realpath('run.py')) + '/data/test.csv'))

if target == "all":
    data = datacleaning.cleaning(os.path.join(os.path.dirname(
        os.path.realpath('run.py')) + '/data/allegations_raw.csv'))

In [ ]:
data.isna().sum()

In [ ]:
data.head()

In [ ]:
train, test = train_test_split(data, train_size=0.8)

## Missingness Creation

We want the attribute with missingness to have around the same proportion of missingness for each type. This is because we don't want the amount of missingness to be a confounding factor in our results.

In [ ]:
t = train.copy()
mcar = data_generation.mcar(t, 'substantiated')

In [ ]:
mcar['substantiated'].isna().sum() / mcar.shape[0]

In [ ]:
mcar = mcar.dropna(subset = 'substantiated')

In [ ]:
t = train.copy()
mar = data_generation.mar(t, 'substantiated', 'complainant_ethnicity', 0.3)

In [ ]:
mar['substantiated'].isna().sum() / mar.shape[0]

In [ ]:
mar = mar.dropna(subset = 'substantiated')

In [ ]:
t = train.copy()
nmar = data_generation.nmar(t, 'substantiated', 0.3)

In [ ]:
nmar['substantiated'].isna().sum() / nmar.shape[0]

In [ ]:
nmar = nmar.dropna(subset = 'substantiated')

Now we will "handle" the missingness by dropping missing values.

## Applying Fairness Notions


In [ ]:
cat = ["complainant_ethnicity", "complainant_age_incident", "allegation", "contact_reason"]

### Calculating fairnes notions for No Missingness At All

In [ ]:
#storing fairness notions for no missingness
no_missing_fairness = []
no_missing = model_perform.model(train, test, cat)
no_missing_fairness.append(no_missing)


In [ ]:
no_missing

### Fairness notions for NMAR

In [ ]:
train_nmar, test_nmar = train_test_split(nmar, test_size=0.2)

In [ ]:
nmar_fairness = []
nmar_model = model_perform.model_missing(train_nmar, test_nmar, cat)
nmar_fairness.append(nmar_model)

### Fairness notions for MCAR

In [ ]:
train_mcar, test_mcar = train_test_split(mcar, test_size=0.2)


In [ ]:
mcar_fairness = []
mcar_model = model_perform.model_missing(train_mcar, test_mcar, cat)
mcar_fairness.append(mcar_model)

### Fairness notions for MAR

In [ ]:
train_mar, test_mar = train_test_split(mar, test_size=0.2)


In [ ]:
mar_fairness = []
mar_model = model_perform.model_missing(train_mar, test_mar, cat)
mar_fairness.append(mar_model)

In [ ]:
#put our fairness statistics into arrays for future usage
acc = [no_missing_fairness[0][0],nmar_fairness[0][0],mcar_fairness[0][0],mar_fairness[0][0]]
par= [no_missing_fairness[0][1],nmar_fairness[0][1],mcar_fairness[0][1],mar_fairness[0][1]]
odds= [no_missing_fairness[0][2],nmar_fairness[0][2],mcar_fairness[0][2],mar_fairness[0][2]]
opp = [no_missing_fairness[0][3],nmar_fairness[0][3],mcar_fairness[0][3],mar_fairness[0][3]]

## Visualizing Our Results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
labels = ['No Missingess,', 'NMAR', 'MCAR', 'MAR']

In [ ]:
plt.figure(figsize = (20, 10))
plt.title('Statistical Parities')
plt.xlabel('Missingness Type')
plt.ylabel('Statistical Parotities')
plt.ylim(min(par) - 0.1, max(par) + 0.1)
plt.plot(labels, par, marker='.', markersize = 20)

In [ ]:
plt.figure(figsize = (20, 10))
plt.title('Equality of Odds')
plt.xlabel('Missingness Type')
plt.ylabel('Equality of Odds')
plt.ylim(min(odds)-0.1,max(odds) + 0.1)
plt.plot(labels, odds, marker='.', markersize = 20)

In [ ]:
plt.figure(figsize = (20, 10))
plt.title('Equality of Opportunity')
plt.xlabel('Missingness Type')
plt.ylabel('Equality of Opportunity')
plt.ylim(min(opp)-0.1,max(opp) + 0.1)
plt.plot(labels, opp, marker='.', markersize = 20)

In [ ]:
plt.figure(figsize = (20, 10))
plt.title('Accuracies')
plt.xlabel('Missingness Type')
plt.ylabel('Accuracies')
plt.ylim(min(acc)-0.1,max(acc) + 0.1)
plt.plot(labels, acc, marker='.', markersize = 20)